In [1]:
from random import random, seed
from math import exp

##### 1. Initialize Network

In [2]:
def initialize_network(n_inputs, n_hidden, n_outputs):
    # Create a list named 'network' in which I contain hidden and output layer
    network = []
    
    # Create a hidden layer
    hidden_layer = [{'weights' : [random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    
    # Create a output layer
    output_layer = [{'weights' : [random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    
    return network

##### 2. Forward Propagate Inputs

##### 2-1. Neuron Activation

In [3]:
# Calculate neuron activaton for an input
def activate(weights, inputs):
    # Include the bias for calculation
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation    

##### 2-2. Neuron Transfer

In [5]:
# Transfer neuron activation
def transfer(activation):
    # Sigmoid(Logistic) function is applied
    return 1.0 / (1.0 + exp(-activation))

##### 2-3. Forward Propagate input to a network output

In [6]:
def forward_propagate(network, row):
    # Create a variable 'input' to forward propagate the neurons in the input layer to the hidden layer
    # and save the values in row to the variable 'input'.
    input = row
    
    # Loop the network list to access the layers sequentially
    for layer in network:
        # Create a variable 'new_inputs' to contain the values that result from activate and transfer.
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], input)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
            
        # We need this to be used as the input to the next layer
        input = new_inputs
        
    return network